In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('data/data.csv', sep=';')
df = df.drop(columns=['Number', 'Closed_date', 'Merge_date', 'Diff_MergeLastModify', 'Create_date', 'Diff_CreateNow'])

print(df.info())
df.head(5)

In [ ]:
# separar os dados em faceis/difíceis
dfEasy = df.loc[(df['Situation'] == 'Closed') & (df['Nivel'] == 'Easy')]
dfDifficult = df.loc[(df['Situation'] == 'Closed') & (df['Nivel'] == 'Difficult')]

In [ ]:
y = dfEasy['Nivel']
X = dfEasy.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.043, shuffle=True)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.044, shuffle=True)

dfTeste = pd.DataFrame(np.concatenate((dfDifficult.values, X_test), axis=0), columns=df.columns) 
dfValid =  pd.DataFrame(np.concatenate((dfDifficult.values, X_val), axis=0), columns=df.columns)

len(X_train), len(X_test), len(X_val)

In [ ]:
columns = ['Imports', 'Changed_files', 'Changes', 'Deletion','Aditions', 'Size_body', 'PR_comments', 'PR_size_comments','Review_comments', 'Review_size_comments', 'Issue_comments','Issue_size_comments', 'Qtd_labels', 'Commits']

def showHistogram(df, label):
    print('Histograma para os dados de ', label)
    
    for idx, column in enumerate(columns):
        for columnAgn in columns[idx+1:]:
            # sns.histplot(data=df[column], bins=10, x=column, hue='Nivel', multiple='dodge', shrink=0.8)
            sns.scatterplot(data=df, x=column, y=columnAgn, hue='Nivel')
            plt.show()

# for idx, data in enumerate(dfTeste.loc[dfTeste['Nivel'] == 'Difficult']['Deletion']):
#     if dfValid.loc[dfValid['Nivel'] == 'Difficult']['Deletion'][idx] != data:
#         print('IDX: ', idx)

showHistogram(dfTeste, 'teste:')
showHistogram(dfValid, 'validação:')

In [ ]:
df.groupby('Nivel').describe()

In [ ]:
# columns = [['Commits', 'Issue_comments'], ['Qtd_labels', 'Commits'], ['Qtd_labels', 'Issue_comments']]
columns = ['Imports', 'Changed_files', 'Changes', 'Deletion','Aditions', 'Size_body', 'PR_comments', 'PR_size_comments','Review_comments', 'Review_size_comments', 'Issue_comments','Issue_size_comments', 'Qtd_labels', 'Commits']

def showBoxplot(df, label):
    print('Bloxplot para os dados de ', label)
    
    for idx, column in enumerate(columns):
        for columnAgn in columns[idx+1:]:
            sns.boxplot(data=df, x=column, y=columnAgn, hue='Nivel')
            plt.show()

showBoxplot(dfTeste, 'teste:')
showBoxplot(dfValid, 'validação:')

In [174]:
columns = ['Imports', 'Changed_files', 'Changes', 'Deletion', 'Aditions', 'Size_body', 'PR_comments', 'PR_size_comments', 'Review_comments', 'Review_size_comments', 'Issue_comments', 'Issue_size_comments', 'Qtd_labels', 'Commits']
labels = ['Difficult', 'Easy']

def median(df):
    for label in labels:
        for column in columns:
            nivelDiff = df.loc[df['Nivel'] == label, column].values.tolist()
            tamDiff = len(nivelDiff)
            data = []

            for values in nivelDiff:
                data.append(values/tamDiff)
            
            df.loc[df['Nivel'] == label, column] = data

median(dfTeste)
median(dfValid)